In [1]:
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd
import numpy as np
import warnings
import random
import pickle
import time
import os

warnings.filterwarnings('ignore')

In [2]:
fake = Faker()
np.random.seed(42)
random.seed(42)

# === CONFIGURATION ===
TARGET_FINAL_SIZE_GB = 0.1
BYTES_PER_GB = 1024 ** 3



# === REALISTIC CONSTANTS ===

# Income distribution parameters (log-normal)
INCOME_PARAMS = {
    'mean_log': 10.5,  # log of median income ~$36,000
    'std_log': 0.7,    # creates realistic income spread
    'min_income': 15000,
    'max_income': 1000000
}

# Credit score distribution (beta-scaled)
CREDIT_SCORE_PARAMS = {
    'alpha': 2.5,
    'beta': 1.5,
    'min_score': 300,
    'max_score': 850
}

# Age demographics (mixture of normals)
AGE_DEMOGRAPHICS = {
    'young_adults': {'mean': 28, 'std': 5, 'weight': 0.25},
    'middle_age': {'mean': 42, 'std': 8, 'weight': 0.45},
    'senior': {'mean': 62, 'std': 8, 'weight': 0.30}
}

# Realistic loan parameters by product type
LOAN_PRODUCTS = {
    'Personal Loan': {
        'min_amount': 1000, 'max_amount': 50000,
        'min_term': 12, 'max_term': 60,
        'base_rate': 8.0, 'risk_premium': 15.0
    },
    'Auto Loan': {
        'min_amount': 5000, 'max_amount': 80000,
        'min_term': 24, 'max_term': 84,
        'base_rate': 4.0, 'risk_premium': 10.0
    },
    'Mortgage': {
        'min_amount': 50000, 'max_amount': 1000000,
        'min_term': 120, 'max_term': 360,
        'base_rate': 3.0, 'risk_premium': 4.0
    },
    'Student Loan': {
        'min_amount': 5000, 'max_amount': 100000,
        'min_term': 60, 'max_term': 240,
        'base_rate': 4.5, 'risk_premium': 3.0
    },
    'Credit Card': {
        'min_amount': 500, 'max_amount': 50000,
        'min_term': 0, 'max_term': 0,  # Revolving credit
        'base_rate': 15.0, 'risk_premium': 10.0
    },
    'Business Loan': {
        'min_amount': 10000, 'max_amount': 500000,
        'min_term': 12, 'max_term': 120,
        'base_rate': 6.0, 'risk_premium': 12.0
    }
}

# Employment sectors with realistic income distributions
EMPLOYMENT_SECTORS = {
    'Technology': {'income_multiplier': 1.4, 'stability': 0.85, 'growth_rate': 0.08},
    'Finance': {'income_multiplier': 1.5, 'stability': 0.80, 'growth_rate': 0.06},
    'Healthcare': {'income_multiplier': 1.3, 'stability': 0.90, 'growth_rate': 0.05},
    'Education': {'income_multiplier': 0.9, 'stability': 0.95, 'growth_rate': 0.03},
    'Retail': {'income_multiplier': 0.7, 'stability': 0.60, 'growth_rate': 0.02},
    'Manufacturing': {'income_multiplier': 1.0, 'stability': 0.75, 'growth_rate': 0.03},
    'Government': {'income_multiplier': 1.1, 'stability': 0.95, 'growth_rate': 0.04},
    'Construction': {'income_multiplier': 1.1, 'stability': 0.65, 'growth_rate': 0.04},
    'Services': {'income_multiplier': 0.8, 'stability': 0.70, 'growth_rate': 0.03}
}

# Transaction patterns by category
TRANSACTION_PATTERNS = {
    'Essential': {
        'Groceries': {'freq_per_month': 8, 'amount_range': (50, 300), 'variance': 0.2},
        'Utilities': {'freq_per_month': 3, 'amount_range': (100, 400), 'variance': 0.1},
        'Housing': {'freq_per_month': 1, 'amount_range': (800, 3000), 'variance': 0.05},
        'Transportation': {'freq_per_month': 10, 'amount_range': (20, 200), 'variance': 0.3},
        'Healthcare': {'freq_per_month': 0.5, 'amount_range': (50, 500), 'variance': 0.5}
    },
    'Discretionary': {
        'Dining': {'freq_per_month': 6, 'amount_range': (20, 150), 'variance': 0.4},
        'Entertainment': {'freq_per_month': 4, 'amount_range': (20, 200), 'variance': 0.5},
        'Shopping': {'freq_per_month': 3, 'amount_range': (50, 500), 'variance': 0.6},
        'Travel': {'freq_per_month': 0.2, 'amount_range': (200, 2000), 'variance': 0.7}
    }
}

# Risk factors and their weights
RISK_FACTORS = {
    'credit_score': {'weight': 0.35, 'threshold': 650},
    'dti_ratio': {'weight': 0.25, 'threshold': 0.43},
    'payment_history': {'weight': 0.20, 'threshold': 0.95},
    'employment_stability': {'weight': 0.10, 'threshold': 0.7},
    'income_verification': {'weight': 0.10, 'threshold': 0.9}
}

# === MORE REALISTIC ROW COUNTS ===
BASE_ROW_COUNTS = {
    'persons': 100_000,              # 100K unique customers
    'person_employment': 150_000,     # 1.5 jobs per person average
    'applications': 250_000,          # 2.5 applications per person average
    'application_financial': 250_000, # One per application
    'credit_scoring': 250_000,        # One per application
    'loans': 150_000,                 # 60% approval rate
    'loan_payments': 2_000_000,       # ~25 payments per loan
    'guarantors': 8_000,              # 5.3% of loans have guarantors
    'collateral': 15_000,             # 10% of loans have collateral
    'transactions': 10_000_000,       # 100 transactions per person per month
    'transactions_p2p': 2_000_000,    # 20 P2P per person per month
    'wof': 2_000,                     # 1.3% default rate
    'related_persons': 10_000         # 10% have related persons
}

# === TABLE SIZE ESTIMATION ===
ESTIMATED_ROW_SIZES = {
    'persons': 400,
    'person_employment': 350,
    'applications': 400,
    'application_financial': 600,
    'credit_scoring': 800,
    'loans': 450,
    'loan_payments': 350,
    'guarantors': 300,
    'collateral': 450,
    'transactions': 300,
    'transactions_p2p': 300,
    'wof': 250,
    'related_persons': 200
}

TARGET_FINAL_SIZE_GB_WITHOUT_WLOB = TARGET_FINAL_SIZE_GB / 1.8

# Calculate scaling factor
estimated_total_bytes = sum(BASE_ROW_COUNTS[table] * ESTIMATED_ROW_SIZES[table]
                          for table in BASE_ROW_COUNTS)
scaling_factor = (TARGET_FINAL_SIZE_GB_WITHOUT_WLOB * BYTES_PER_GB) / estimated_total_bytes

# Scale all tables proportionally
TARGET_ROW_COUNTS = {
    table: max(1, int(BASE_ROW_COUNTS[table] * scaling_factor))
    for table in BASE_ROW_COUNTS
}

TARGET_ROW_COUNTS

{'persons': 1212,
 'person_employment': 1818,
 'applications': 3030,
 'application_financial': 3030,
 'credit_scoring': 3030,
 'loans': 1818,
 'loan_payments': 24240,
 'guarantors': 96,
 'collateral': 181,
 'transactions': 121203,
 'transactions_p2p': 24240,
 'wof': 24,
 'related_persons': 121}

In [3]:
# === HELPER FUNCTIONS ===

def generate_correlated_values(n, correlation_matrix, means, stds):
    """Generate correlated random variables"""
    # Generate uncorrelated standard normal variables
    uncorrelated = np.random.randn(n, len(means))

    # Apply Cholesky decomposition to create correlation
    L = np.linalg.cholesky(correlation_matrix)
    correlated = uncorrelated @ L.T

    # Scale and shift to desired means and standard deviations
    scaled = correlated * stds + means

    return scaled

def generate_realistic_ages(n):
    """Generate realistic age distribution - OPTIMIZED"""
    # Vectorized approach
    groups = list(AGE_DEMOGRAPHICS.keys())
    probs = [demo['weight'] for demo in AGE_DEMOGRAPHICS.values()]

    # Generate all group choices at once
    chosen_groups = np.random.choice(groups, size=n, p=probs)

    # Pre-compute means and stds for all groups
    group_means = {group: AGE_DEMOGRAPHICS[group]['mean'] for group in groups}
    group_stds = {group: AGE_DEMOGRAPHICS[group]['std'] for group in groups}

    # Vectorized age generation
    ages = np.zeros(n)
    for group in groups:
        mask = chosen_groups == group
        count = np.sum(mask)
        if count > 0:
            ages[mask] = np.random.normal(group_means[group], group_stds[group], count)

    # Clip ages in one operation
    return np.clip(ages, 18, 85).astype(int)

def generate_income_distribution(n):
    """Generate realistic income distribution - ALREADY OPTIMIZED"""
    log_incomes = np.random.normal(INCOME_PARAMS['mean_log'],
                                  INCOME_PARAMS['std_log'], n)
    incomes = np.exp(log_incomes)
    incomes = np.clip(incomes, INCOME_PARAMS['min_income'],
                     INCOME_PARAMS['max_income'])
    return incomes

def generate_credit_scores(n, incomes, ages):
    """Generate credit scores correlated with income and age - ALREADY OPTIMIZED"""
    # All operations are already vectorized using numpy
    beta_scores = np.random.beta(CREDIT_SCORE_PARAMS['alpha'],
                                CREDIT_SCORE_PARAMS['beta'], n)
    base_scores = CREDIT_SCORE_PARAMS['min_score'] + beta_scores * (
        CREDIT_SCORE_PARAMS['max_score'] - CREDIT_SCORE_PARAMS['min_score']
    )

    income_factor = np.log(incomes) / np.log(INCOME_PARAMS['max_income'])
    income_adjustment = income_factor * 100

    age_factor = np.clip((ages - 18) / 50, 0, 1)
    age_adjustment = age_factor * 50

    final_scores = base_scores + income_adjustment + age_adjustment
    noise = np.random.normal(0, 30, n)
    final_scores = np.clip(final_scores + noise,
                          CREDIT_SCORE_PARAMS['min_score'],
                          CREDIT_SCORE_PARAMS['max_score'])

    return final_scores.astype(int)

def generate_persons(n):
    """Generate ultra high-quality person data with realistic correlations - OPTIMIZED"""
    # Generate correlated demographic features
    ages = generate_realistic_ages(n)
    incomes = generate_income_distribution(n)
    credit_scores = generate_credit_scores(n, incomes, ages)

    # Vectorized education levels
    education_choices = ['High School', 'Some College', 'Bachelor', 'Master', 'PhD']

    # Create income brackets
    income_brackets = np.digitize(incomes, [30000, 60000, 100000, 200000])

    # Pre-defined probabilities for each bracket
    education_probs = {
        0: [0.50, 0.30, 0.15, 0.04, 0.01],
        1: [0.30, 0.30, 0.30, 0.08, 0.02],
        2: [0.15, 0.20, 0.40, 0.20, 0.05],
        3: [0.05, 0.10, 0.40, 0.35, 0.10],
        4: [0.02, 0.08, 0.30, 0.40, 0.20]
    }

    # Vectorized education generation
    education_levels = np.empty(n, dtype=object)
    for bracket in range(5):
        mask = income_brackets == bracket
        count = np.sum(mask)
        if count > 0:
            education_levels[mask] = np.random.choice(
                education_choices,
                size=count,
                p=education_probs[bracket]
            )

    # Vectorized gender generation
    genders = np.random.choice(['M', 'F', 'O'], size=n, p=[0.49, 0.49, 0.02])

    # Vectorized marital status
    age_brackets = np.digitize(ages, [25, 35, 50])
    marital_probs = {
        0: [0.80, 0.15, 0.04, 0.01],
        1: [0.40, 0.50, 0.08, 0.02],
        2: [0.20, 0.60, 0.15, 0.05],
        3: [0.15, 0.50, 0.20, 0.15]
    }

    marital_statuses = np.empty(n, dtype=object)
    marital_choices = ['Single', 'Married', 'Divorced', 'Widowed']
    for bracket in range(4):
        mask = age_brackets == bracket
        count = np.sum(mask)
        if count > 0:
            marital_statuses[mask] = np.random.choice(
                marital_choices,
                size=count,
                p=marital_probs[bracket]
            )

    # Vectorized home ownership
    ownership_prob = np.minimum(0.9, 0.1 + (ages - 18) / 100 + incomes / 500000)
    owns_home = np.random.random(n) < ownership_prob

    # Vectorized home status selection
    home_ownerships = np.empty(n, dtype=object)
    home_ownerships[~owns_home] = 'Rent'

    # For those who own
    high_income_owners = owns_home & (incomes > 100000)
    low_income_owners = owns_home & (incomes <= 100000)

    home_ownerships[high_income_owners] = np.random.choice(
        ['Own', 'Mortgage'],
        size=np.sum(high_income_owners),
        p=[0.3, 0.7]
    )
    home_ownerships[low_income_owners] = np.random.choice(
        ['Own', 'Mortgage'],
        size=np.sum(low_income_owners),
        p=[0.1, 0.9]
    )

    # Geographic distribution
    states = ['CA', 'TX', 'FL', 'NY', 'PA', 'IL', 'OH', 'GA', 'NC', 'MI']
    state_weights = [0.15, 0.12, 0.10, 0.10, 0.08, 0.08, 0.07, 0.07, 0.07, 0.06]
    states_selected = np.random.choice(states, size=n, p=state_weights/np.sum(state_weights))

    # Vectorized date generation
    current_date = datetime.now()
    dob_dates = [current_date - timedelta(days=int(age*365.25)) for age in ages]
    customer_since_dates = [fake.date_between(start_date='-15y', end_date='today') for _ in range(n)]
    created_dates = [fake.date_time_between(start_date='-5y', end_date='now') for _ in range(n)]
    updated_dates = [fake.date_time_between(start_date='-1y', end_date='now') for _ in range(n)]

    # Vectorized dependent calculation
    married_mask = marital_statuses == 'Married'
    dependents = np.where(married_mask,
                         np.random.poisson(1.5, n),
                         np.random.poisson(0.3, n))

    # Create the dataframe
    persons_df = pd.DataFrame({
        'personid': range(1, n + 1),
        'first_name': [fake.first_name() for _ in range(n)],
        'last_name': [fake.last_name() for _ in range(n)],
        'email': [f"{fake.user_name()}@{fake.free_email_domain()}" for _ in range(n)],
        'phone': [fake.phone_number()[:15] for _ in range(n)],
        'ssn': [fake.ssn() for _ in range(n)],
        'date_of_birth': dob_dates,
        'age': ages,
        'gender': genders,
        'marital_status': marital_statuses,
        'education_level': education_levels,
        'annual_income': incomes.round(2),
        'credit_score': credit_scores,
        'address_street': [fake.street_address() for _ in range(n)],
        'address_city': [fake.city() for _ in range(n)],
        'address_state': states_selected,
        'address_zip': [fake.postcode()[:5] for _ in range(n)],
        'home_ownership': home_ownerships,
        'years_at_address': np.random.exponential(5, n).round(1).clip(0, 30),
        'dependents': dependents,
        'customer_since': customer_since_dates,
        'customer_segment': pd.qcut(incomes + credit_scores * 10,
                                   q=4,
                                   labels=['Basic', 'Standard', 'Premium', 'Elite']),
        'risk_category': pd.qcut(credit_scores,
                                q=4,
                                labels=['High Risk', 'Medium Risk', 'Low Risk', 'Very Low Risk']),
        'is_active': np.random.choice([True, False], size=n, p=[0.92, 0.08]),
        'preferred_contact': np.random.choice(['Email', 'Phone', 'Mail', 'SMS'],
                                            size=n, p=[0.45, 0.25, 0.10, 0.20]),
        'occupation_category': np.random.choice(['Professional', 'Technical', 'Service',
                                               'Sales', 'Administrative', 'Labor', 'Other'],
                                              size=n),
        'created_at': created_dates,
        'updated_at': updated_dates
    })

    return persons_df

def generate_person_employment(n, persons_df):
    """Generate employment history with realistic career progressions - OPTIMIZED"""
    # Pre-calculate employment counts for all persons
    employment_counts = np.random.poisson(1.5, len(persons_df)).clip(0, 5)

    # Calculate career start ages and work years for all persons at once
    career_start_ages = persons_df['education_level'].map({
        'High School': 18,
        'Some College': 20,
        'Bachelor': 22,
        'Master': 24,
        'PhD': 27
    }).fillna(18)

    work_years = np.maximum(0, persons_df['age'] - career_start_ages)

    # Pre-allocate lists for batch processing
    employment_records = []
    current_time = datetime.now()

    # Vectorized sector selection probabilities
    education_level_probs = {
        ('Master', 'PhD'): [0.25, 0.20, 0.15, 0.15, 0.05, 0.05, 0.10, 0.02, 0.03],
        ('Bachelor',): [0.20, 0.15, 0.10, 0.05, 0.10, 0.15, 0.10, 0.05, 0.10],
        ('High School', 'Some College'): [0.05, 0.05, 0.10, 0.05, 0.20, 0.20, 0.05, 0.15, 0.15]
    }

    # Process persons in batches for memory efficiency
    for idx, person in persons_df.iterrows():
        num_jobs = employment_counts[idx]
        work_year = work_years[idx]

        if work_year == 0 or num_jobs == 0:
            continue

        years_per_job = work_year / max(1, num_jobs)

        # Determine sector probabilities
        for edu_group, probs in education_level_probs.items():
            if person['education_level'] in edu_group:
                sector_probs = probs
                break
        else:
            sector_probs = education_level_probs[('High School', 'Some College')]

        # Generate all sectors at once for this person's jobs
        sectors = np.random.choice(list(EMPLOYMENT_SECTORS.keys()),
                                 size=min(num_jobs, n - len(employment_records)),
                                 p=sector_probs)

        # Vectorized position level determination
        experiences = work_year - np.arange(num_jobs) * years_per_job
        position_levels = np.where(experiences < 2, 'Junior',
                          np.where(experiences < 5, '',
                          np.where(experiences < 10, 'Senior',
                          np.where(experiences < 15, 'Lead', 'Director'))))

        # Generate employment records for this person
        for job_idx in range(min(num_jobs, n - len(employment_records))):
            sector = sectors[job_idx]
            sector_info = EMPLOYMENT_SECTORS[sector]

            # Calculate salary
            if job_idx == 0:  # Current job
                base_salary = person['annual_income']
            else:
                years_ago = job_idx * years_per_job
                growth_factor = (1 + sector_info['growth_rate']) ** (-years_ago)
                base_salary = person['annual_income'] * growth_factor

            sector_salary = base_salary * sector_info['income_multiplier']

            # Calculate dates
            if job_idx == 0:
                start_date = current_time - timedelta(days=int(years_per_job * 365))
                end_date = None
                is_current = True
                employment_status = 'Full-time'
            else:
                end_date = current_time - timedelta(days=int((job_idx - 0.5) * years_per_job * 365))
                start_date = end_date - timedelta(days=int(years_per_job * 365))
                is_current = False
                employment_status = np.random.choice(['Full-time', 'Part-time', 'Contract'],
                                                   p=[0.8, 0.1, 0.1])

            position_title = f"{position_levels[job_idx]} {fake.job()}".strip()

            employment_records.append({
                'employment_id': len(employment_records) + 1,
                'personid': person['personid'],
                'employer_name': fake.company(),
                'employer_industry': sector,
                'position_title': position_title,
                'employment_status': employment_status,
                'start_date': start_date,
                'end_date': end_date,
                'annual_salary': round(sector_salary, 2),
                'monthly_salary': round(sector_salary / 12, 2),
                'years_in_role': round(years_per_job, 1),
                'total_experience': round(experiences[job_idx], 1),
                'is_current': is_current,
                'performance_rating': np.random.choice(['Exceeds', 'Meets', 'Below'],
                                                     p=[0.2, 0.7, 0.1]),
                'termination_reason': None if is_current else np.random.choice(
                    ['Better Opportunity', 'Layoff', 'Resignation', 'Contract End'],
                    p=[0.6, 0.1, 0.2, 0.1]
                ),
                'employer_size': np.random.choice(['Small', 'Medium', 'Large', 'Enterprise'],
                                                p=[0.3, 0.3, 0.2, 0.2]),
                'verification_status': np.random.choice(['Verified', 'Pending', 'Unverified'],
                                                      p=[0.7, 0.2, 0.1]),
                'created_at': fake.date_time_between(start_date='-3y', end_date='now'),
                'updated_at': fake.date_time_between(start_date='-1y', end_date='now')
            })

            if len(employment_records) >= n:
                break

        if len(employment_records) >= n:
            break

    return pd.DataFrame(employment_records[:n])

def generate_applications(n, persons_df, employment_df):
    """Generate historical loan applications with approved amounts - ALREADY OPTIMIZED"""
    # The existing code is already highly vectorized
    # Let's make just a few minor improvements

    # Sample persons for applications
    person_sample = persons_df.sample(n, replace=True).reset_index(drop=True)

    # Get current employment information
    current_employment = employment_df[employment_df['is_current'] == True].set_index('personid')

    # Vectorized has employment check
    has_employment = person_sample['personid'].isin(current_employment.index).values

    # Vectorized product selection
    high_income_mask = (person_sample['annual_income'] > 100000) & (person_sample['credit_score'] > 720)
    young_mask = person_sample['age'] < 25
    low_income_mask = person_sample['annual_income'] < 40000

    # Product names and probabilities
    product_names = list(LOAN_PRODUCTS.keys())

    # Vectorized product type selection using numpy choice
    probs_matrix = np.array([
        [0.25, 0.25, 0.2, 0.1, 0.15, 0.05],  # default
        [0.2, 0.2, 0.3, 0.05, 0.15, 0.1],     # high_income
        [0.3, 0.2, 0.05, 0.35, 0.1, 0.0],     # young
        [0.4, 0.1, 0.05, 0.1, 0.3, 0.05]      # low_income
    ])

    # Create condition array
    conditions = np.zeros(n, dtype=int)
    conditions[high_income_mask] = 1
    conditions[young_mask] = 2
    conditions[low_income_mask] = 3

    # Generate all product types at once
    product_types = np.array([
        np.random.choice(product_names, p=probs_matrix[cond])
        for cond in conditions
    ])

    # Vectorized calculations for all loan parameters
    product_info_dict = {pt: LOAN_PRODUCTS[pt] for pt in product_names}

    # Vectorized max amount calculation
    max_amounts = np.zeros(n)
    for pt in product_names:
        mask = product_types == pt
        if np.any(mask):
            if pt == 'Mortgage':
                max_amounts[mask] = np.minimum(
                    product_info_dict[pt]['max_amount'],
                    person_sample.loc[mask, 'annual_income'] * 5
                )
            elif pt == 'Auto Loan':
                max_amounts[mask] = np.minimum(
                    product_info_dict[pt]['max_amount'],
                    person_sample.loc[mask, 'annual_income'] * 0.5
                )
            elif pt == 'Credit Card':
                max_amounts[mask] = np.minimum(
                    product_info_dict[pt]['max_amount'],
                    person_sample.loc[mask, 'annual_income'] * 0.3
                )
            else:
                max_amounts[mask] = np.minimum(
                    product_info_dict[pt]['max_amount'],
                    person_sample.loc[mask, 'annual_income'] * 1.5
                )

    # Vectorized min amounts and requested amounts
    min_amounts = np.array([product_info_dict[pt]['min_amount'] for pt in product_types])
    requested_amounts = np.random.uniform(min_amounts, max_amounts)

    # Vectorized term months
    term_months = np.array([
        0 if pt == 'Credit Card' else
        np.random.randint(product_info_dict[pt]['min_term'],
                         product_info_dict[pt]['max_term'] + 1)
        for pt in product_types
    ])

    # Vectorized interest rates
    base_rates = np.array([product_info_dict[pt]['base_rate'] for pt in product_types])
    risk_premiums = np.array([product_info_dict[pt]['risk_premium'] for pt in product_types])
    credit_factors = (850 - person_sample['credit_score'].values) / 550
    interest_rates = base_rates + risk_premiums * credit_factors

    # Vectorized channel selection
    young_small_loan = (person_sample['age'] < 35) & (requested_amounts < 50000)
    large_loan = requested_amounts > 100000

    # Create probability matrix for channels
    channel_probs = np.zeros((n, 4))  # Online, Branch, Mobile, Partner

    # Set default probabilities
    channel_probs[:, 0] = 0.4  # Online
    channel_probs[:, 1] = 0.3  # Branch
    channel_probs[:, 2] = 0.2  # Mobile
    channel_probs[:, 3] = 0.1  # Partner

    # Adjust for young small loans
    channel_probs[young_small_loan, 0] = 0.6
    channel_probs[young_small_loan, 1] = 0.1
    channel_probs[young_small_loan, 2] = 0.25
    channel_probs[young_small_loan, 3] = 0.05

    # Adjust for large loans
    channel_probs[large_loan, 0] = 0.2
    channel_probs[large_loan, 1] = 0.6
    channel_probs[large_loan, 2] = 0.1
    channel_probs[large_loan, 3] = 0.1

    # Generate channels using vectorized choice
    channels = np.array(['Online', 'Branch', 'Mobile', 'Partner'])[
        np.array([np.random.choice(4, p=probs) for probs in channel_probs])
    ]

    # Vectorized purposes
    purposes = np.where(
        product_types == 'Personal Loan',
        np.random.choice(['Debt Consolidation', 'Home Improvement', 'Medical',
                         'Major Purchase', 'Other'], size=n),
        np.where(
            product_types == 'Business Loan',
            np.random.choice(['Working Capital', 'Equipment', 'Expansion',
                            'Inventory', 'Other'], size=n),
            product_types
        )
    )

    # Vectorized dates
    current_time = pd.Timestamp.now()
    app_dates = pd.to_datetime([
        fake.date_time_between(start_date='-2y', end_date='now')
        for _ in range(n)
    ])

    # Vectorized approval logic
    approval_probs = np.select(
        [
            has_employment & (person_sample['credit_score'] > 650),
            has_employment & (person_sample['credit_score'] > 600),
            has_employment
        ],
        [0.8, 0.6, 0.3],
        default=0.1
    )

    decisions = np.random.random(n) < approval_probs
    decision_status = np.where(decisions, 'Approved', 'Declined')

    # Vectorized approved amounts
    approved_amounts = np.where(
        decisions,
        np.where(
            person_sample['credit_score'] < 650,
            requested_amounts * np.random.uniform(0.7, 0.9, n),
            requested_amounts * np.random.uniform(0.9, 1.0, n)
        ),
        0
    )

    # Vectorized date calculations
    decision_dates = app_dates + pd.to_timedelta(np.random.randint(1, 10, n), unit='D')
    funding_dates = np.where(
        decisions,
        decision_dates + pd.to_timedelta(np.random.randint(1, 15, n), unit='D'),
        pd.NaT
    )

    # Vectorized decline reasons
    decline_reasons = np.select(
        [
            ~decisions & (person_sample['credit_score'] < 600),
            ~decisions & ~has_employment,
            ~decisions & (person_sample['annual_income'] < 30000),
            ~decisions
        ],
        ['Low Credit Score', 'No Employment', 'Insufficient Income', 'Credit Policy'],
        default=None
    )

    # Pre-generate all IDs at once
    uw_ids = [f"UW{np.random.randint(1000, 9999)}" for _ in range(n)]
    br_ids = np.where(channels == 'Branch',
                      [f"BR{np.random.randint(100, 999)}" for _ in range(n)],
                      None)
    mkt_mask = np.random.random(n) < 0.3
    mkt_ids = np.where(mkt_mask,
                       [f"MKT{np.random.randint(1000, 9999)}" for _ in range(n)],
                       None)

    # Create DataFrame all at once
    return pd.DataFrame({
        'appid': np.arange(1, n + 1),
        'personid': person_sample['personid'].values,
        'product_type': product_types,
        'requested_amount': requested_amounts.round(2),
        'approved_amount': approved_amounts.round(2),
        'term_months': term_months,
        'interest_rate': interest_rates.round(3),
        'application_date': app_dates,
        'decision_date': decision_dates,
        'funding_date': funding_dates,
        'application_status': 'Completed',
        'decision_status': decision_status,
        'channel': channels,
        'purpose': purposes,
        'has_coborrower': np.random.choice([True, False], size=n, p=[0.1, 0.9]),
        'has_collateral': np.isin(product_types, ['Mortgage', 'Auto Loan', 'Business Loan']),
        'application_score': np.random.randint(300, 850, n),
        'approval_probability': approval_probs,
        'decline_reasons': decline_reasons,
        'underwriter_id': uw_ids,
        'branch_id': br_ids,
        'marketing_campaign_id': mkt_ids,
        'referral_source': np.random.choice(['Direct', 'Referral', 'Advertisement', 'Partner'],
                                          size=n, p=[0.6, 0.2, 0.15, 0.05]),
        'created_at': app_dates,
        'updated_at': decision_dates
    })

def generate_application_financial(n, applications_df, persons_df, employment_df):
    """Generate detailed financial information for each application - FULLY FIXED"""

    # Create merged dataframe for efficient access
    merged_df = applications_df.merge(persons_df, on='personid', how='left')

    # Get current employment information
    current_employment = employment_df[employment_df['is_current'] == True].set_index('personid')

    # Vectorized employment data extraction
    has_employment = merged_df['personid'].isin(current_employment.index).values
    monthly_employment_income = np.zeros(n)
    employment_stability = np.zeros(n)

    # Extract employment data in bulk
    employed_persons = merged_df['personid'][has_employment].values
    for idx, person_id in enumerate(employed_persons):
        if person_id in current_employment.index:
            emp = current_employment.loc[person_id]
            if isinstance(emp, pd.DataFrame):
                emp = emp.iloc[0]
            pos = np.where(merged_df['personid'] == person_id)[0][0]
            monthly_employment_income[pos] = emp['monthly_salary']
            employment_stability[pos] = EMPLOYMENT_SECTORS[emp['employer_industry']]['stability']

    annual_employment_income = monthly_employment_income * 12

    # Vectorized other income calculation
    has_other_income = np.random.random(n) < 0.3
    other_income_types = np.where(
        has_other_income,
        np.random.choice(['Investment', 'Rental', 'Side Business', 'Alimony', 'Other'], size=n),
        None
    )

    # FIXED: Handle potential NaN or negative values
    base_income = merged_df['annual_income'].fillna(50000).values
    other_income_amount = np.where(
        has_other_income,
        np.random.exponential(np.maximum(base_income * 0.1, 1000)),
        0
    )

    total_annual_income = annual_employment_income + other_income_amount
    total_monthly_income = total_annual_income / 12

    # Ensure no zero or negative values for income
    total_monthly_income = np.maximum(total_monthly_income, 1000)

    # Vectorized asset calculations
    age_values = merged_df['age'].fillna(30).values
    credit_scores = merged_df['credit_score'].fillna(650).values

    asset_multiplier = 0.1 + (age_values - 18) / 100 + credit_scores / 1000
    total_assets = total_annual_income * asset_multiplier * np.random.uniform(0.5, 2.0, n)
    liquid_assets = total_assets * np.random.uniform(0.2, 0.6, n)

    # Vectorized retirement assets
    retirement_assets = np.select(
        [
            age_values < 30,
            age_values < 50,
            age_values >= 50
        ],
        [
            total_annual_income * np.random.uniform(0, 0.5, n),
            total_annual_income * np.random.uniform(0.5, 3, n),
            total_annual_income * np.random.uniform(2, 10, n)
        ],
        default=0
    )

    # FIXED: Handle home ownership with fillna
    home_ownership = merged_df['home_ownership'].fillna('Rent').values
    mortgage_mask = home_ownership == 'Mortgage'

    mortgage_debt = np.where(
        mortgage_mask,
        total_annual_income * np.random.uniform(1, 4, n),
        0
    )
    mortgage_payment = np.where(mortgage_mask, mortgage_debt / (30 * 12), 0)

    credit_card_debt = total_annual_income * np.random.uniform(0, 0.3, n)

    auto_loan_mask = np.random.random(n) < 0.5
    auto_loan_debt = np.where(
        auto_loan_mask,
        total_annual_income * np.random.uniform(0, 0.4, n),
        0
    )

    # FIXED: Handle education level with fillna
    education_level = merged_df['education_level'].fillna('High School').values
    student_loan_mask = (age_values < 40) & (education_level != 'High School')
    student_loan_debt = np.where(
        student_loan_mask,
        total_annual_income * np.random.uniform(0, 0.5, n),
        0
    )

    other_debt = total_annual_income * np.random.uniform(0, 0.2, n)
    total_debt = mortgage_debt + credit_card_debt + auto_loan_debt + student_loan_debt + other_debt

    # Vectorized monthly payments
    credit_card_payment = credit_card_debt * 0.03
    auto_loan_payment = np.where(auto_loan_debt > 0, auto_loan_debt / 60, 0)
    student_loan_payment = np.where(student_loan_debt > 0, student_loan_debt / 120, 0)
    other_debt_payment = np.where(other_debt > 0, other_debt / 36, 0)

    total_monthly_debt_payments = (
        mortgage_payment + credit_card_payment +
        auto_loan_payment + student_loan_payment +
        other_debt_payment
    )

    # FIXED: Safe division for DTI ratio
    dti_ratio = np.where(
        (total_monthly_income > 0) & np.isfinite(total_monthly_income),
        np.minimum(total_monthly_debt_payments / total_monthly_income, 0.99),
        0.99
    )

    # Vectorized expense calculations
    housing_expense = np.where(
        mortgage_mask,
        mortgage_payment,
        total_monthly_income * np.random.uniform(0.2, 0.35, n)
    )

    utilities_expense = np.random.uniform(150, 400, n)
    transportation_expense = auto_loan_payment + np.random.uniform(100, 300, n)
    food_expense = np.random.uniform(300, 800, n)
    insurance_expense = np.random.uniform(100, 500, n)
    other_expenses = np.random.uniform(200, 1000, n)

    total_monthly_expenses = (
        housing_expense + utilities_expense + transportation_expense +
        food_expense + insurance_expense + other_expenses
    )

    # FIXED: Safe division for housing expense ratio
    housing_expense_ratio = np.where(
        (total_monthly_income > 0) & np.isfinite(total_monthly_income),
        np.minimum(housing_expense / total_monthly_income, 0.99),
        0.99
    )

    disposable_income = total_monthly_income - total_monthly_expenses - total_monthly_debt_payments

    # FIXED: Safe division for savings rate
    savings_rate = np.where(
        (total_monthly_income > 0) & np.isfinite(total_monthly_income),
        np.clip(disposable_income / total_monthly_income, -1, 1),
        0
    )

    # Vectorized flags
    bankruptcy_flag = np.random.random(n) < (0.05 * np.where(dti_ratio > 0.5, 2, 1))

    # FIXED: Safe comparison for credit scores
    credit_score_values = merged_df['credit_score'].fillna(650).values
    default_history = np.random.random(n) < (0.1 * np.where(credit_score_values < 600, 2, 1))

    # Vectorized verification statuses
    income_verified = np.random.choice([True, False], size=n, p=[0.8, 0.2])
    assets_verified = np.random.choice([True, False], size=n, p=[0.7, 0.3])
    employment_verified = np.random.choice([True, False], size=n, p=[0.85, 0.15])

    # Create DataFrame
    return pd.DataFrame({
        'financial_id': np.arange(1, n + 1),
        'appid': applications_df['appid'],
        'personid': applications_df['personid'],

        # Income
        'employment_income': annual_employment_income.round(2),
        'other_income': other_income_amount.round(2),
        'other_income_type': other_income_types,
        'total_annual_income': total_annual_income.round(2),
        'total_monthly_income': total_monthly_income.round(2),

        # Assets
        'total_assets': total_assets.round(2),
        'liquid_assets': liquid_assets.round(2),
        'retirement_assets': retirement_assets.round(2),
        'real_estate_assets': (total_assets - liquid_assets - retirement_assets).round(2),

        # Debts
        'mortgage_debt': mortgage_debt.round(2),
        'credit_card_debt': credit_card_debt.round(2),
        'auto_loan_debt': auto_loan_debt.round(2),
        'student_loan_debt': student_loan_debt.round(2),
        'other_debt': other_debt.round(2),
        'total_debt': total_debt.round(2),

        # Monthly payments
        'mortgage_payment': mortgage_payment.round(2),
        'credit_card_payment': credit_card_payment.round(2),
        'auto_loan_payment': auto_loan_payment.round(2),
        'student_loan_payment': student_loan_payment.round(2),
        'other_debt_payment': other_debt_payment.round(2),
        'total_monthly_debt_payments': total_monthly_debt_payments.round(2),

        # Expenses
        'housing_expense': housing_expense.round(2),
        'utilities_expense': utilities_expense.round(2),
        'transportation_expense': transportation_expense.round(2),
        'food_expense': food_expense.round(2),
        'insurance_expense': insurance_expense.round(2),
        'other_expenses': other_expenses.round(2),
        'total_monthly_expenses': total_monthly_expenses.round(2),

        # Ratios and flags
        'debt_to_income_ratio': dti_ratio.round(3),
        'housing_expense_ratio': housing_expense_ratio.round(3),
        'disposable_income': disposable_income.round(2),
        'savings_rate': savings_rate.round(3),
        'bankruptcy_flag': bankruptcy_flag,
        'default_history': default_history,
        'employment_stability_score': employment_stability,

        # Verification status
        'income_verified': income_verified,
        'assets_verified': assets_verified,
        'employment_verified': employment_verified,

        'financial_date': applications_df['application_date'],
        'created_at': applications_df['application_date'],
        'updated_at': applications_df['application_date'] + pd.to_timedelta(np.random.randint(1, 7, n), unit='D')
    })

def generate_credit_scoring(n, applications_df, persons_df, financial_df):
    """Generate comprehensive credit scoring data - OPTIMIZED"""

    # Pre-merge dataframes to avoid repeated lookups
    merged_app_person = applications_df.merge(persons_df, on='personid', how='left')
    merged_app_financial = applications_df.merge(financial_df, on='appid', how='left')

    # Extract arrays for vectorized operations
    base_credit_scores = merged_app_person['credit_score'].values
    ages = merged_app_person['age'].values
    total_debts = merged_app_financial['total_debt'].values
    credit_card_debts = merged_app_financial['credit_card_debt'].values
    employment_stability = merged_app_financial['employment_stability_score'].values
    dti_ratios = merged_app_financial['debt_to_income_ratio'].values
    requested_amounts = applications_df['requested_amount'].values
    has_collateral = applications_df['has_collateral'].values
    product_types = applications_df['product_type'].values
    app_dates = applications_df['application_date'].values

    # Vectorized bureau information
    bureau_names = np.random.choice(['Equifax', 'Experian', 'TransUnion'], size=n)
    bureau_scores = np.clip(base_credit_scores + np.random.randint(-20, 21, n), 300, 850)

    # Fixed vectorized credit history calculations
    credit_history_length = np.zeros(n, dtype=int)

    # For ages < 21
    young_mask = ages < 21
    credit_history_length[young_mask] = np.random.randint(0, 3, np.sum(young_mask))

    # For ages >= 21
    adult_mask = ~young_mask
    min_history = np.maximum(0, (ages[adult_mask] - 18) // 2)
    max_history = ages[adult_mask] - 18

    # Ensure min is always less than max
    valid_range_mask = min_history < max_history

    # For cases where min < max, generate random values
    credit_history_length[adult_mask][valid_range_mask] = np.array([
        np.random.randint(min_val, max_val)
        for min_val, max_val in zip(min_history[valid_range_mask], max_history[valid_range_mask])
    ])

    # For cases where min >= max, use min value
    credit_history_length[adult_mask][~valid_range_mask] = min_history[~valid_range_mask]

    total_accounts = np.where(
        ages < 21,
        np.random.randint(0, 5, n),
        np.random.poisson(credit_history_length * 0.7)
    )

    open_accounts = (total_accounts * np.random.uniform(0.5, 0.9, n)).astype(int)
    closed_accounts = total_accounts - open_accounts

    # Vectorized delinquency information
    delinquent_accounts = np.zeros(n, dtype=int)
    collections_count = np.zeros(n, dtype=int)
    bankruptcies_count = np.zeros(n, dtype=int)
    max_delinquency = np.zeros(n, dtype=int)

    # High credit score (>700)
    high_score_mask = base_credit_scores > 700

    # Medium credit score (650-700)
    med_score_mask = (base_credit_scores > 650) & (base_credit_scores <= 700)
    delinquent_accounts[med_score_mask] = np.random.choice([0, 1], size=np.sum(med_score_mask), p=[0.9, 0.1])
    collections_count[med_score_mask] = np.random.choice([0, 1], size=np.sum(med_score_mask), p=[0.95, 0.05])
    max_delinquency[med_score_mask] = np.random.choice([0, 30, 60], size=np.sum(med_score_mask), p=[0.8, 0.15, 0.05])

    # Low credit score (<=650)
    low_score_mask = base_credit_scores <= 650
    delinquent_accounts[low_score_mask] = np.random.poisson(0.5, np.sum(low_score_mask))
    collections_count[low_score_mask] = np.random.poisson(0.3, np.sum(low_score_mask))
    bankruptcies_count[low_score_mask] = np.random.choice([0, 1], size=np.sum(low_score_mask), p=[0.9, 0.1])
    max_delinquency[low_score_mask] = np.random.choice([0, 30, 60, 90, 120], size=np.sum(low_score_mask), p=[0.5, 0.2, 0.15, 0.1, 0.05])

    # Vectorized credit utilization
    credit_utilization = np.where(
        total_debts > 0,
        np.minimum(credit_card_debts / (total_debts * 0.3), 1.0),
        0
    )

    # Vectorized payment history
    on_time_payments = np.where(
        delinquent_accounts == 0,
        total_accounts * 12 * np.minimum(credit_history_length, 7),
        (total_accounts * 12 * np.minimum(credit_history_length, 7) * np.random.uniform(0.7, 0.95, n)).astype(int)
    )

    late_payments = np.where(
        delinquent_accounts == 0,
        0,
        (total_accounts * 12 * np.minimum(credit_history_length, 7) - on_time_payments).astype(int)
    )

    # Vectorized recent inquiries
    recent_inquiries = np.where(
        product_types == 'Credit Card',
        np.random.poisson(2, n),
        np.random.poisson(1, n)
    )

    # Vectorized risk assessment
    risk_scores = np.full(n, 100)
    risk_factors = [[] for _ in range(n)]

    # Apply credit score penalties
    low_credit_mask = base_credit_scores < 600
    fair_credit_mask = (base_credit_scores >= 600) & (base_credit_scores < 650)

    risk_scores[low_credit_mask] -= 30
    risk_scores[fair_credit_mask] -= 15

    # Apply DTI penalties
    high_dti_mask = dti_ratios > 0.43
    elevated_dti_mask = (dti_ratios > 0.36) & (dti_ratios <= 0.43)

    risk_scores[high_dti_mask] -= 20
    risk_scores[elevated_dti_mask] -= 10

    # Apply employment stability penalties
    low_stability_mask = employment_stability < 0.7
    risk_scores[low_stability_mask] -= 15

    # Apply payment history penalties
    late_payment_mask = late_payments > 0
    risk_scores[late_payment_mask] -= 20

    # Apply bankruptcy penalties
    bankruptcy_mask = bankruptcies_count > 0
    risk_scores[bankruptcy_mask] -= 40

    # Calculate application scores
    application_scores = (
        0.4 * base_credit_scores +
        0.3 * (100 - dti_ratios * 100) +
        0.2 * employment_stability * 100 +
        0.1 * risk_scores
    ).astype(int)

    # Vectorized fraud scoring
    fraud_indicators = (
        (recent_inquiries > 5).astype(int) +
        (merged_app_person['years_at_address'] < 1).astype(int) +
        (~merged_app_financial['income_verified']).astype(int) +
        (requested_amounts > merged_app_person['annual_income']).astype(int)
    )

    fraud_scores = np.minimum(1000, fraud_indicators * 200 + np.random.exponential(50, n))

    # Vectorized probability of default
    pd_base = np.full(n, 0.05)
    pd_base[base_credit_scores < 600] *= 4
    pd_base[(base_credit_scores >= 600) & (base_credit_scores < 650)] *= 2
    pd_base[dti_ratios > 0.43] *= 2
    pd_base[delinquent_accounts > 0] *= 1.5

    probability_default = np.minimum(1.0, pd_base * np.random.uniform(0.8, 1.2, n))

    # Vectorized loss given default
    lgd = np.where(
        has_collateral,
        np.random.beta(2, 8, n),  # Mean around 0.2
        np.random.beta(6, 4, n)   # Mean around 0.6
    )

    # Expected loss calculation
    ead = requested_amounts
    expected_loss = ead * probability_default * lgd

    # Vectorized verification statuses
    identity_verification = np.random.choice(['Verified', 'Pending', 'Failed'], size=n, p=[0.9, 0.08, 0.02])

    # Vectorized recommendations
    recommendations = np.where(
        (application_scores > 650) & (risk_scores > 60) & (fraud_scores < 500),
        'Approve',
        'Decline'
    )

    confidence_scores = np.random.uniform(0.7, 0.99, n)

    # Create risk factors strings
    risk_factors_list = []
    for i in range(n):
        factors = []
        if base_credit_scores[i] < 600:
            factors.append('Low Credit Score')
        elif base_credit_scores[i] < 650:
            factors.append('Fair Credit Score')

        if dti_ratios[i] > 0.43:
            factors.append('High Debt-to-Income Ratio')
        elif dti_ratios[i] > 0.36:
            factors.append('Elevated Debt-to-Income Ratio')

        if employment_stability[i] < 0.7:
            factors.append('Employment Instability')

        if late_payments[i] > 0:
            factors.append('Payment History Issues')

        if bankruptcies_count[i] > 0:
            factors.append('Prior Bankruptcy')

        risk_factors_list.append(','.join(factors) if factors else None)

    # Vectorized dates
    credit_report_dates = pd.to_datetime(app_dates) - pd.to_timedelta(np.random.randint(1, 30, n), unit='D')
    score_dates = pd.to_datetime(app_dates)
    created_dates = pd.to_datetime(app_dates)
    updated_dates = pd.to_datetime(app_dates) + pd.to_timedelta(np.random.randint(1, 24, n), unit='h')

    # Create DataFrame
    return pd.DataFrame({
        'scoring_id': np.arange(1, n + 1),
        'appid': applications_df['appid'],
        'personid': applications_df['personid'],

        # Credit bureau information
        'bureau_name': bureau_names,
        'bureau_score': bureau_scores,
        'credit_score': base_credit_scores,
        'credit_report_date': credit_report_dates,

        # Credit history
        'credit_history_length_months': credit_history_length * 12,
        'total_accounts': total_accounts,
        'open_accounts': open_accounts,
        'closed_accounts': closed_accounts,
        'delinquent_accounts': delinquent_accounts,
        'collections_count': collections_count,
        'bankruptcies_count': bankruptcies_count,
        'foreclosures_count': np.where(base_credit_scores > 600, 0,
                                      np.random.choice([0, 1], size=n, p=[0.95, 0.05])),

        # Credit utilization
        'total_credit_limit': np.where(credit_utilization > 0,
                                      total_debts / credit_utilization,
                                      merged_app_financial['total_annual_income'] * 0.5).round(2),
        'total_credit_balance': total_debts.round(2),
        'credit_utilization_ratio': credit_utilization.round(3),

        # Payment history
        'on_time_payments': on_time_payments,
        'late_payments': late_payments,
        'missed_payments': (late_payments * 0.2).astype(int),
        'max_delinquency_days': max_delinquency,

        # Recent activity
        'recent_inquiries_6m': recent_inquiries,
        'new_accounts_6m': np.random.poisson(0.5, n),
        'recent_balance_increase': np.random.choice([True, False], size=n, p=[0.3, 0.7]),

        # Risk assessment
        'application_score': application_scores,
        'risk_score': risk_scores,
        'risk_factors': risk_factors_list,
        'fraud_score': fraud_scores.astype(int),
        'fraud_flags': fraud_indicators,

        # Probability metrics
        'probability_default': probability_default.round(4),
        'loss_given_default': lgd.round(4),
        'exposure_at_default': ead.round(2),
        'expected_loss': expected_loss.round(2),

        # Verification scores
        'income_verification_status': np.where(merged_app_financial['income_verified'],
                                              'Verified', 'Unverified'),
        'employment_verification_status': np.where(merged_app_financial['employment_verified'],
                                                  'Verified', 'Unverified'),
        'identity_verification_status': identity_verification,

        # Decision recommendation
        'recommendation': recommendations,
        'confidence_score': confidence_scores.round(3),

        'score_date': score_dates,
        'created_at': created_dates,
        'updated_at': updated_dates
    })

def generate_loans(n, applications_df):
    """Generate loan records with guaranteed output - OPTIMIZED"""
    # Create copies to avoid modifying original data
    apps = applications_df.copy()

    # Check if we have any approved applications already
    decision_column = None
    for col in ['decision_status', 'status', 'application_status']:
        if col in apps.columns:
            decision_column = col
            break

    if decision_column:
        approved_apps = apps[apps[decision_column] == 'Approved']
    else:
        approved_apps = pd.DataFrame()

    # If no approved applications, create them
    if len(approved_apps) == 0:
        # Take a sample of applications and mark them as approved
        if len(apps) > 0:
            sample_size = min(n, len(apps))
            approved_apps = apps.sample(sample_size).copy()
            approved_apps['decision_status'] = 'Approved'

            if 'approved_amount' not in approved_apps.columns:
                if 'requested_amount' in approved_apps.columns:
                    approved_apps['approved_amount'] = approved_apps['requested_amount']
                else:
                    approved_apps['approved_amount'] = np.random.uniform(10000, 100000, len(approved_apps))
        else:
            # Create synthetic approved applications
            approved_apps = pd.DataFrame({
                'appid': range(1, n + 1),
                'personid': np.random.randint(1, 1000, n),
                'product_type': np.random.choice(['Personal Loan', 'Auto Loan', 'Mortgage', 'Business Loan'], n),
                'requested_amount': np.random.uniform(10000, 100000, n),
                'approved_amount': np.random.uniform(10000, 100000, n),
                'term_months': np.random.choice([12, 24, 36, 48, 60], n),
                'interest_rate': np.random.uniform(5, 15, n),
                'application_date': pd.date_range(end=datetime.now(), periods=n, freq='-1D')[::-1],
                'decision_status': 'Approved',
                'channel': np.random.choice(['Online', 'Branch', 'Mobile'], n),
                'purpose': np.random.choice(['Home Improvement', 'Debt Consolidation', 'Major Purchase'], n)
            })

    # Limit to requested number
    approved_apps = approved_apps.head(n).reset_index(drop=True)

    # Generate loan details using vectorized operations
    current_date = datetime.now()
    n_loans = len(approved_apps)

    # Extract and prepare data columns
    approved_amounts = approved_apps['approved_amount'].values if 'approved_amount' in approved_apps.columns else \
                      approved_apps['requested_amount'].values if 'requested_amount' in approved_apps.columns else \
                      np.random.uniform(10000, 100000, n_loans)

    app_dates = pd.to_datetime(approved_apps['application_date']) if 'application_date' in approved_apps.columns else \
                current_date - pd.to_timedelta(np.random.randint(30, 365, n_loans), unit='D')

    term_months = approved_apps['term_months'].values if 'term_months' in approved_apps.columns else \
                  np.random.choice([12, 24, 36, 48, 60], n_loans)

    interest_rates = approved_apps['interest_rate'].values if 'interest_rate' in approved_apps.columns else \
                     np.random.uniform(5, 15, n_loans)

    # Vectorized date calculations
    decision_dates = app_dates + pd.to_timedelta(np.random.randint(1, 10, n_loans), unit='D')
    funding_dates = decision_dates + pd.to_timedelta(np.random.randint(1, 15, n_loans), unit='D')
    maturity_dates = funding_dates + pd.to_timedelta(term_months * 30, unit='D')

    # Calculate loan age and status
    loan_age_days = (current_date - funding_dates).dt.days

    # Vectorized loan status determination
    loan_status = np.where(
        loan_age_days < 0, 'Pending',
        np.where(
            loan_age_days < 90, 'Current',
            np.where(
                loan_age_days > term_months * 30, 'Paid Off',
                np.where(
                    np.random.random(n_loans) < 0.05,
                    np.where(loan_age_days <= 180, 'Default', 'Charged Off'),
                    'Current'
                )
            )
        )
    )

    # Vectorized payment calculations
    monthly_payments = np.where(term_months > 0, approved_amounts / term_months, 0)
    payments_made = np.minimum(np.maximum(loan_age_days // 30, 0), term_months)

    # Vectorized balance calculations
    current_balances = np.where(
        loan_status == 'Paid Off',
        0,
        np.maximum(approved_amounts - monthly_payments * payments_made, 0)
    )

    total_paid = np.where(
        loan_status == 'Paid Off',
        approved_amounts * (1 + interest_rates / 100 * term_months / 12),
        monthly_payments * payments_made
    )

    # Vectorized days past due calculation
    days_past_due = np.where(
        np.isin(loan_status, ['Default', 'Charged Off']),
        np.random.randint(30, 180, n_loans),
        np.where(
            (loan_status == 'Current') & (np.random.random(n_loans) < 0.1),
            np.random.randint(1, 29, n_loans),
            0
        )
    )

    # Create DataFrame
    loans_df = pd.DataFrame({
        'loanid': np.arange(1, n_loans + 1),
        'appid': approved_apps['appid'].values if 'appid' in approved_apps.columns else np.arange(n_loans),
        'personid': approved_apps['personid'].values if 'personid' in approved_apps.columns else np.random.randint(1, 1000, n_loans),
        'product_type': approved_apps['product_type'].values if 'product_type' in approved_apps.columns else 'Personal Loan',
        'original_amount': approved_amounts.round(2),
        'current_balance': current_balances.round(2),
        'interest_rate': interest_rates.round(3),
        'term_months': term_months,
        'monthly_payment': monthly_payments.round(2),
        'origination_date': funding_dates,
        'maturity_date': maturity_dates,
        'loan_status': loan_status,
        'days_past_due': days_past_due,
        'total_paid': total_paid.round(2),
        'loan_purpose': approved_apps['purpose'].values if 'purpose' in approved_apps.columns else 'General',
        'created_at': funding_dates,
        'updated_at': current_date
    })

    return loans_df

def generate_loan_payments(n, loans_df):
    """Generate detailed loan payment history"""
    payments = []

    for _, loan in loans_df.iterrows():
        if loan['term_months'] == 0:  # Skip credit cards
            continue

        # Calculate payment schedule
        loan_amount = loan['original_amount']
        monthly_rate = loan['interest_rate'] / 100 / 12
        num_payments = loan['term_months']

        # Monthly payment calculation (PMT formula)
        if monthly_rate > 0:
            monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate)**num_payments) / ((1 + monthly_rate)**num_payments - 1)
        else:
            monthly_payment = loan_amount / num_payments

        # Generate payment history
        current_balance = loan_amount
        loan_start = loan['origination_date']
        current_date = datetime.now()

        # Determine payment behavior pattern
        if loan['loan_status'] in ['Default', 'Charged Off']:
            payment_pattern = 'Delinquent'
        elif loan['days_past_due'] > 0:
            payment_pattern = 'Occasionally Late'
        else:
            payment_pattern = 'On Time'

        for month in range(num_payments):
            due_date = loan_start + timedelta(days=30 * (month + 1))

            # Skip future payments
            if due_date > current_date:
                break

            # Determine payment behavior
            if payment_pattern == 'On Time':
                payment_delay = np.random.choice([-5, -3, -1, 0, 1], p=[0.1, 0.2, 0.3, 0.3, 0.1])
            elif payment_pattern == 'Occasionally Late':
                payment_delay = np.random.choice([-3, 0, 3, 7, 15], p=[0.1, 0.4, 0.3, 0.15, 0.05])
            else:  # Delinquent
                if month < num_payments * 0.3:  # Early payments were okay
                    payment_delay = np.random.choice([0, 3, 7], p=[0.6, 0.3, 0.1])
                else:  # Later payments became problematic
                    payment_delay = np.random.choice([15, 30, 60, 90], p=[0.2, 0.3, 0.3, 0.2])

            # Convert numpy.int64 to regular Python int for timedelta
            payment_delay = int(payment_delay)
            payment_date = due_date + timedelta(days=payment_delay)

            # Skip if payment is in the future
            if payment_date > current_date:
                break

            # Calculate principal and interest portions
            interest_portion = current_balance * monthly_rate
            principal_portion = monthly_payment - interest_portion

            # Simulate payment variations
            if payment_delay > 5:
                late_fee = monthly_payment * 0.05
                payment_status = 'Late'
            else:
                late_fee = 0
                payment_status = 'On Time'

            # Sometimes partial payments
            if payment_pattern == 'Delinquent' and month > num_payments * 0.3 and np.random.random() < 0.3:
                actual_payment = monthly_payment * np.random.uniform(0.5, 0.9)
                payment_status = 'Partial'
            else:
                actual_payment = monthly_payment + late_fee

            payments.append({
                'payment_id': len(payments) + 1,
                'loanid': loan['loanid'],
                'payment_date': payment_date,
                'due_date': due_date,
                'payment_amount': round(actual_payment, 2),
                'principal_portion': round(principal_portion, 2),
                'interest_portion': round(interest_portion, 2),
                'late_fee': round(late_fee, 2),
                'payment_method': np.random.choice(['Auto-debit', 'Transfer', 'Online', 'Check', 'Cash'],
                                                 p=[0.5, 0.2, 0.2, 0.05, 0.05]),
                'payment_status': payment_status,
                'remaining_balance': round(current_balance - principal_portion, 2)
            })

            current_balance -= principal_portion

            if len(payments) >= n:
                break

        if len(payments) >= n:
            break

    return pd.DataFrame(payments[:n])

def generate_transactions(n, persons_df):
    """Generate realistic transaction patterns more efficiently"""
    transactions = []
    current_date = datetime.now()

    # Pre-define merchant names
    merchant_names = {
        'Groceries': ['Walmart', 'Kroger', 'Safeway', 'Whole Foods', 'Target'],
        'Dining': ['Starbucks', 'McDonald\'s', 'Chipotle', 'Local Restaurant', 'Pizza Place'],
        'Shopping': ['Amazon', 'Best Buy', 'Macy\'s', 'Home Depot', 'Local Store'],
        'Transportation': ['Shell', 'Chevron', 'Uber', 'Lyft', 'Transit Authority'],
        'Utilities': ['Electric Company', 'Water Company', 'Gas Company', 'Internet Provider'],
        'Entertainment': ['Netflix', 'AMC Theaters', 'Local Cinema', 'Concert Venue'],
        'Healthcare': ['Local Pharmacy', 'Doctor\'s Office', 'Hospital', 'Dental Office'],
        'Housing': ['Mortgage Company', 'Property Management', 'HOA'],
        'Travel': ['Airlines', 'Hotels', 'Booking.com', 'Airbnb']
    }

    # Pre-compute transaction counts by customer segment
    segment_to_tx_count = {
        'Elite': (40, 80),
        'Premium': (30, 50),
        'Standard': (20, 35),
        'Basic': (10, 25)
    }

    essential_categories = list(TRANSACTION_PATTERNS['Essential'].keys())
    discretionary_categories = list(TRANSACTION_PATTERNS['Discretionary'].keys())

    # Process in batches for better vectorization
    batch_size = min(1000, n)
    remaining = n
    transaction_id_counter = 0

    # Process person by person (while keeping track of total transactions)
    for _, person in persons_df.iterrows():
        # Skip if we've generated enough transactions
        if remaining <= 0:
            break

        # Determine monthly transaction count based on segment
        segment = person['customer_segment']
        min_tx, max_tx = segment_to_tx_count.get(segment, (10, 25))
        monthly_transactions = np.random.randint(min_tx, max_tx)

        # Calculate person's income factor once
        income_factor = person['annual_income'] / 50000  # Normalize to median income
        income_factor = 0.5 + 0.5 * min(income_factor, 3)

        # Generate batches of transactions for this person
        person_months = min(12, (remaining + monthly_transactions - 1) // monthly_transactions)

        for month_offset in range(person_months):
            # Skip if we've generated enough transactions
            if remaining <= 0:
                break

            month_start = current_date - timedelta(days=30 * month_offset)

            # Determine actual transaction count for this month (respect remaining count)
            month_tx_count = min(monthly_transactions, remaining)
            remaining -= month_tx_count

            # Generate vectors for all month's transactions at once
            # 1. Transaction dates
            days_offset = np.random.randint(0, 30, size=month_tx_count)
            transaction_dates = [month_start - timedelta(days=int(days)) for days in days_offset]

            # 2. Categories
            category_random = np.random.random(size=month_tx_count)
            is_essential = category_random < 0.6

            categories = np.empty(month_tx_count, dtype=object)
            category_types = np.empty(month_tx_count, dtype=object)

            # Set essential categories
            essential_count = np.sum(is_essential)
            if essential_count > 0:
                essential_indices = np.random.randint(0, len(essential_categories), size=essential_count)
                categories[is_essential] = [essential_categories[i] for i in essential_indices]
                category_types[is_essential] = 'Essential'

            # Set discretionary categories
            discretionary_count = month_tx_count - essential_count
            if discretionary_count > 0:
                discretionary_indices = np.random.randint(0, len(discretionary_categories), size=discretionary_count)
                categories[~is_essential] = [discretionary_categories[i] for i in discretionary_indices]
                category_types[~is_essential] = 'Discretionary'

            # 3. Transaction amounts
            base_amounts = np.zeros(month_tx_count)
            variances = np.zeros(month_tx_count)

            # Calculate amounts for each category
            for i, (cat_type, category) in enumerate(zip(category_types, categories)):
                pattern = TRANSACTION_PATTERNS[cat_type][category]
                min_amount, max_amount = pattern['amount_range']
                base_amounts[i] = np.random.uniform(min_amount, max_amount)
                variances[i] = pattern['variance']

            # Apply income factor and variance
            amounts = base_amounts * income_factor
            variance_factors = 1 + np.random.normal(0, variances)
            amounts = amounts * variance_factors
            amounts = np.maximum(5, amounts)  # Minimum transaction amount
            amounts = np.round(amounts, 2)

            # 4. Transaction types
            tx_types = np.empty(month_tx_count, dtype=object)
            high_value_mask = amounts > 1000

            # For transactions > $1000
            high_value_count = np.sum(high_value_mask)
            if high_value_count > 0:
                high_value_types = np.random.choice(
                    ['ACH', 'Wire', 'Card'],
                    size=high_value_count,
                    p=[0.4, 0.3, 0.3]
                )
                tx_types[high_value_mask] = high_value_types

            # For transactions <= $1000
            low_value_count = month_tx_count - high_value_count
            if low_value_count > 0:
                low_value_types = np.random.choice(
                    ['Card', 'ACH', 'Cash'],
                    size=low_value_count,
                    p=[0.7, 0.2, 0.1]
                )
                tx_types[~high_value_mask] = low_value_types

            # 5. Merchant names
            merchant_array = np.empty(month_tx_count, dtype=object)
            for i, category in enumerate(categories):
                possible_merchants = merchant_names.get(category, ['Generic Merchant'])
                merchant_array[i] = np.random.choice(possible_merchants)

            # 6. Transaction status and decline reasons
            status_random = np.random.random(size=month_tx_count)
            statuses = np.full(month_tx_count, 'Completed', dtype=object)
            statuses[status_random < 0.02] = 'Declined'  # 2% declined
            statuses[np.logical_and(status_random >= 0.02, status_random < 0.03)] = 'Pending'  # 1% pending

            decline_reasons = np.full(month_tx_count, None, dtype=object)
            declined_mask = statuses == 'Declined'

            if np.any(declined_mask):
                decline_options = ['Insufficient Funds', 'Fraud Alert', 'Invalid Card']
                decline_values = np.random.choice(decline_options, size=np.sum(declined_mask))
                decline_reasons[declined_mask] = decline_values

            # 7. Channels
            channels = np.random.choice(
                ['Online', 'POS', 'ATM', 'Mobile'],
                size=month_tx_count,
                p=[0.3, 0.4, 0.1, 0.2]
            )

            # 8. International flag
            is_international = np.random.random(size=month_tx_count) < 0.05

            # 9. Post dates (0-2 days after transaction date)
            post_date_offsets = np.random.randint(0, 3, size=month_tx_count)
            post_dates = [
                tx_date + timedelta(days=int(offset))
                for tx_date, offset in zip(transaction_dates, post_date_offsets)
            ]

            # 10. Update times (1-60 minutes after creation)
            update_minute_offsets = np.random.randint(1, 60, size=month_tx_count)
            updated_at_times = [
                tx_date + timedelta(minutes=int(minutes))
                for tx_date, minutes in zip(transaction_dates, update_minute_offsets)
            ]

            # Generate all transactions for this month
            for i in range(month_tx_count):
                transaction_id_counter += 1

                transactions.append({
                    'transaction_id': transaction_id_counter,
                    'personid': person['personid'],
                    'transaction_date': transaction_dates[i],
                    'post_date': post_dates[i],
                    'transaction_type': tx_types[i],
                    'transaction_category': categories[i],
                    'merchant_name': merchant_array[i],
                    'merchant_category': categories[i],
                    'amount': amounts[i],
                    'currency': 'USD',
                    'transaction_status': statuses[i],
                    'decline_reason': decline_reasons[i],
                    'channel': channels[i],
                    'location_city': person['address_city'],
                    'location_state': person['address_state'],
                    'location_country': 'US',
                    'is_international': is_international[i],
                    'description': f"{merchant_array[i]} - {categories[i]}",
                    'created_at': transaction_dates[i],
                    'updated_at': updated_at_times[i]
                })

    return pd.DataFrame(transactions)

def generate_transactions_p2p(n, persons_df):
    """Generate P2P transaction records more efficiently"""
    # Constants
    p2p_purposes = ['Rent Split', 'Dinner Split', 'Gift', 'Loan Repayment', 'Utilities Split', 'Other']
    p2p_platforms = ['Venmo', 'Zelle', 'CashApp', 'PayPal', 'Bank Transfer']

    # Pre-generate all random selections
    person_ids = persons_df['personid'].values
    num_persons = len(person_ids)

    # Generate sender and receiver pairs ensuring they're different
    senders_idx = np.random.randint(0, num_persons, size=n)
    receivers_idx = np.random.randint(0, num_persons, size=n)

    # Make sure senders and receivers are different people
    same_person_mask = senders_idx == receivers_idx
    if np.any(same_person_mask):
        # Shift receivers who are the same as senders
        receivers_idx[same_person_mask] = (receivers_idx[same_person_mask] + 1) % num_persons

    sender_ids = person_ids[senders_idx]
    receiver_ids = person_ids[receivers_idx]

    # Pre-generate all other random values
    current_date = datetime.now()

    # Generate dates as timestamps in seconds
    date_range_end = int(current_date.timestamp())
    date_range_start = int((current_date - timedelta(days=365)).timestamp())
    transaction_timestamps = np.random.randint(date_range_start, date_range_end, size=n)
    transaction_dates = [datetime.fromtimestamp(ts) for ts in transaction_timestamps]

    # Generate purposes and corresponding amounts
    purposes = np.random.choice(p2p_purposes, size=n)

    # Initialize amounts array
    amounts = np.zeros(n)

    # Set amounts based on purpose
    rent_mask = purposes == 'Rent Split'
    amounts[rent_mask] = np.random.uniform(300, 1500, size=np.sum(rent_mask))

    dinner_mask = purposes == 'Dinner Split'
    amounts[dinner_mask] = np.random.uniform(10, 100, size=np.sum(dinner_mask))

    gift_mask = purposes == 'Gift'
    amounts[gift_mask] = np.random.uniform(20, 500, size=np.sum(gift_mask))

    # All other purposes
    other_mask = ~(rent_mask | dinner_mask | gift_mask)
    amounts[other_mask] = np.random.uniform(10, 1000, size=np.sum(other_mask))

    # Round amounts to 2 decimal places
    amounts = np.round(amounts, 2)

    # Generate platforms
    platforms = np.random.choice(p2p_platforms, size=n)

    # Generate transaction statuses
    random_status = np.random.random(size=n)
    statuses = np.full(n, 'Completed', dtype=object)
    statuses[random_status < 0.01] = 'Failed'  # 1% failed
    statuses[np.logical_and(random_status >= 0.01, random_status < 0.06)] = 'Pending'  # 5% pending

    # Generate failure reasons
    failure_reasons = np.full(n, None, dtype=object)
    failed_mask = statuses == 'Failed'
    if np.any(failed_mask):
        failure_options = ['Insufficient Funds', 'Account Closed', 'Limit Exceeded']
        failure_reasons[failed_mask] = np.random.choice(failure_options, size=np.sum(failed_mask))

    # Generate memos
    have_memo_mask = np.random.random(size=n) < 0.7
    memos = np.full(n, None, dtype=object)
    if np.any(have_memo_mask):
        memos[have_memo_mask] = [fake.sentence() for _ in range(np.sum(have_memo_mask))]

    # Generate is_recurring flags
    recurring_eligible = np.isin(purposes, ['Rent Split', 'Utilities Split'])
    random_recurring = np.random.random(size=n) < 0.8
    is_recurring = np.logical_and(recurring_eligible, random_recurring)

    # Generate update times (1-30 minutes after creation)
    # Convert numpy.int32 to Python int to fix the TypeError
    update_minutes = [int(mins) for mins in np.random.randint(1, 31, size=n)]
    updated_at = [date + timedelta(minutes=mins) for date, mins in zip(transaction_dates, update_minutes)]

    # Create the dataframe in one go
    p2p_transactions = {
        'p2p_transaction_id': np.arange(1, n+1),
        'sender_personid': sender_ids,
        'receiver_personid': receiver_ids,
        'transaction_date': transaction_dates,
        'amount': amounts,
        'currency': ['USD'] * n,
        'purpose': purposes,
        'platform': platforms,
        'transaction_status': statuses,
        'failure_reason': failure_reasons,
        'memo': memos,
        'is_recurring': is_recurring,
        'created_at': transaction_dates,
        'updated_at': updated_at
    }

    return pd.DataFrame(p2p_transactions)

def generate_vwlob(end_date=None, loans_df=None):
    """Generate daily loan observation records showing only active loans for each date - OPTIMIZED"""

    if loans_df is None or len(loans_df) == 0:
        return pd.DataFrame()

    # Use current date if no end date specified
    if end_date is None:
        end_date = datetime.now()
    else:
        end_date = pd.to_datetime(end_date)

    # Convert dates to pandas datetime
    origination_dates = pd.to_datetime(loans_df['origination_date'])
    maturity_dates = pd.to_datetime(loans_df['maturity_date'])

    # Find the earliest origination date and latest date to process
    min_date = origination_dates.min()

    # Generate all dates from earliest origination to end_date
    all_dates = pd.date_range(start=min_date, end=end_date, freq='D')

    all_records = []

    # For each date, find which loans are active
    for current_date in all_dates:
        # A loan is active if:
        # 1. Origination date <= current_date
        # 2. Maturity date > current_date OR loan status is not 'Paid Off'
        active_mask = (
            (origination_dates <= current_date) &
            ((maturity_dates > current_date) | (loans_df['loan_status'] != 'Paid Off'))
        )

        active_loans = loans_df[active_mask].copy()

        if len(active_loans) == 0:
            continue

        # Calculate days on book for each active loan
        days_on_book = (current_date - origination_dates[active_mask]).dt.days

        # Calculate current balance for each active loan
        daily_payment = active_loans['monthly_payment'] / 30
        principal_paid = daily_payment * days_on_book * 0.6
        current_balances = np.maximum(0, active_loans['original_amount'] - principal_paid)

        # Calculate overdue days as of current_date
        overdue_days = np.zeros(len(active_loans), dtype=int)

        for idx, (loan_idx, loan) in enumerate(active_loans.iterrows()):
            if loan['loan_status'] in ['Default', 'Charged Off']:
                # For defaulted loans, calculate progression of overdue days
                days_since_origination = (current_date - pd.to_datetime(loan['origination_date'])).days
                days_to_end = (end_date - pd.to_datetime(loan['origination_date'])).days
                overdue_days[idx] = min(180, max(0, loan['days_past_due'] - (days_to_end - days_since_origination)))
            elif loan['loan_status'] == 'Current' and loan['days_past_due'] > 0:
                # Show overdue only if we're in the last 30 days before end_date
                if (end_date - current_date).days <= 30:
                    overdue_days[idx] = loan['days_past_due']
                else:
                    overdue_days[idx] = 0
            else:
                overdue_days[idx] = 0

        # Determine payment status
        payment_status = pd.cut(
            overdue_days,
            bins=[-np.inf, 0, 30, 60, 90, np.inf],
            labels=['Current', '0-30 DPD', '30-60 DPD', '60-90 DPD', 'Default'],
            right=True
        )

        # Create records for this date
        date_records = pd.DataFrame({
            'loanid': active_loans['loanid'].values,
            'appid': active_loans['appid'].values,
            'personid': active_loans['personid'].values,  # Added personid
            'observation_date': current_date.date(),
            'days_on_book': days_on_book.values,
            'months_on_book': (days_on_book.values // 30),
            'current_balance': current_balances.round(2),
            'overdue_days': overdue_days,
            'payment_status': payment_status,
            'ever_delinquent': (overdue_days > 0) | active_loans['loan_status'].isin(['Default', 'Charged Off']),
            'loan_status': active_loans['loan_status'].values,  # Current loan status
            'created_at': current_date + pd.Timedelta(hours=1),
            'updated_at': current_date + pd.Timedelta(hours=1)
        })

        all_records.append(date_records)

    # Combine all records
    if all_records:
        result_df = pd.concat(all_records, ignore_index=True)
        result_df['vwlob_id'] = np.arange(1, len(result_df) + 1)

        # Return with proper column order
        columns_order = [
            'vwlob_id', 'loanid', 'appid', 'personid', 'observation_date',
            'days_on_book', 'months_on_book', 'current_balance', 'overdue_days',
            'payment_status', 'ever_delinquent', 'max_delinquency', 'loan_status',
            'created_at', 'updated_at'
        ]

        return result_df[columns_order]
    else:
        return pd.DataFrame(columns=[
            'vwlob_id', 'loanid', 'appid', 'personid', 'observation_date',
            'days_on_book', 'months_on_book', 'current_balance', 'overdue_days',
            'payment_status', 'ever_delinquent', 'max_delinquency', 'loan_status',
            'created_at', 'updated_at'
        ])

def generate_wof(n, loans_df):
    """Generate write-off records for defaulted loans"""
    wof_records = []

    # Check if loans_df is empty or if loan_status column exists
    if len(loans_df) == 0:
        print("Warning: No loans data available for write-offs")
        return pd.DataFrame()

    # Check what columns are available
    # print(f"Available columns in loans_df: {loans_df.columns.tolist()}")

    # Check if loan_status column exists, if not try alternative column names
    status_column = None
    if 'loan_status' in loans_df.columns:
        status_column = 'loan_status'
    elif 'status' in loans_df.columns:
        status_column = 'status'
    elif 'loanStatus' in loans_df.columns:
        status_column = 'loanStatus'

    if status_column is None:
        print("Warning: No status column found in loans dataframe")
        # Create some synthetic write-offs based on loan age or other criteria
        # For now, just take a random sample of loans
        sample_size = min(n, len(loans_df))
        sample_loans = loans_df.sample(sample_size)
    else:
        # Only process charged off loans
        charged_off_loans = loans_df[loans_df[status_column] == 'Charged Off']

        if len(charged_off_loans) == 0:
            print("Warning: No charged off loans found, creating synthetic write-offs")
            # Take a random sample of loans to create synthetic write-offs
            sample_size = min(n, max(1, int(len(loans_df) * 0.02)))  # 2% of loans
            sample_loans = loans_df.sample(sample_size)
        else:
            sample_loans = charged_off_loans

    # Generate write-off records
    for idx, loan in sample_loans.iterrows():
        if len(wof_records) >= n:
            break

        # Get origination date
        if 'origination_date' in loan:
            origination_date = loan['origination_date']
        elif 'originationDate' in loan:
            origination_date = loan['originationDate']
        elif 'start_date' in loan:
            origination_date = loan['start_date']
        else:
            # Use current date minus some random days if no origination date found
            origination_date = datetime.now() - timedelta(days=np.random.randint(180, 720))

        # Ensure origination_date is datetime
        if not isinstance(origination_date, datetime):
            try:
                origination_date = pd.to_datetime(origination_date)
            except:
                origination_date = datetime.now() - timedelta(days=np.random.randint(180, 720))

        # Write-off typically happens after 180 days of delinquency
        wof_date = origination_date + timedelta(days=180 + np.random.randint(0, 30))

        # Get loan amount
        if 'current_balance' in loan and pd.notna(loan['current_balance']):
            wof_amount = loan['current_balance']
        elif 'original_amount' in loan and pd.notna(loan['original_amount']):
            wof_amount = loan['original_amount'] * np.random.uniform(0.3, 0.8)
        elif 'amount' in loan and pd.notna(loan['amount']):
            wof_amount = loan['amount'] * np.random.uniform(0.3, 0.8)
        else:
            wof_amount = np.random.uniform(5000, 50000)

        # Recovery information
        recovery_rate = np.random.beta(2, 8)  # Most recoveries are low
        expected_recovery = wof_amount * recovery_rate
        actual_recovery = expected_recovery * np.random.uniform(0, 1.2)  # Some variation

        # Get personid
        if 'personid' in loan:
            personid = loan['personid']
        elif 'person_id' in loan:
            personid = loan['person_id']
        elif 'customer_id' in loan:
            personid = loan['customer_id']
        else:
            personid = np.random.randint(1, 10000)

        # Get loanid
        if 'loanid' in loan:
            loanid = loan['loanid']
        elif 'loan_id' in loan:
            loanid = loan['loan_id']
        elif 'id' in loan:
            loanid = loan['id']
        else:
            loanid = idx

        wof_records.append({
            'wof_id': len(wof_records) + 1,
            'loanid': loanid,
            'personid': personid,
            'wof_date': wof_date.date() if hasattr(wof_date, 'date') else wof_date,
            'wof_amount': round(wof_amount, 2),
            'wof_reason': np.random.choice(['Non-Payment', 'Bankruptcy', 'Deceased', 'Fraud', 'Other'],
                                         p=[0.7, 0.15, 0.05, 0.05, 0.05]),
            'expected_recovery': round(expected_recovery, 2),
            'actual_recovery': round(actual_recovery, 2),
            'recovery_status': 'Completed' if actual_recovery > 0 else 'In Progress',
            'collection_agency': np.random.choice(['Internal', 'Agency A', 'Agency B', 'Agency C'],
                                                p=[0.3, 0.3, 0.2, 0.2]),
            'created_at': wof_date,
            'updated_at': wof_date + timedelta(days=np.random.randint(1, 30))
        })

    if len(wof_records) == 0:
        # Return empty dataframe with proper structure
        return pd.DataFrame(columns=['wof_id', 'loanid', 'personid', 'wof_date', 'wof_amount',
                                    'wof_reason', 'expected_recovery', 'actual_recovery',
                                    'recovery_status', 'collection_agency', 'created_at', 'updated_at'])

    return pd.DataFrame(wof_records)

def generate_related_persons(n, persons_df):
    """Generate relationships between persons"""
    related_records = []
    relationship_types = ['Spouse', 'Parent', 'Child', 'Sibling', 'Business Partner', 'Co-applicant']

    # Identify potential relationships
    for _ in range(n):
        person1 = persons_df.sample(1).iloc[0]

        # Find a suitable related person
        if person1['marital_status'] == 'Married' and np.random.random() < 0.7:
            # Find a spouse
            potential_spouses = persons_df[
                (persons_df['personid'] != person1['personid']) &
                (persons_df['marital_status'] == 'Married') &
                (abs(persons_df['age'] - person1['age']) < 15)
            ]
            if len(potential_spouses) > 0:
                person2 = potential_spouses.sample(1).iloc[0]
                relationship_type = 'Spouse'
            else:
                continue
        else:
            # Random relationship
            person2 = persons_df[persons_df['personid'] != person1['personid']].sample(1).iloc[0]
            relationship_type = np.random.choice(relationship_types[1:])  # Exclude spouse

        related_records.append({
            'relationship_id': len(related_records) + 1,
            'personid': person1['personid'],
            'related_personid': person2['personid'],
            'relationship_type': relationship_type,
            'start_date': fake.date_between(start_date='-10y', end_date='-1y'),
            'end_date': None if np.random.random() < 0.8 else fake.date_between(start_date='-1y', end_date='today'),
            'is_active': np.random.random() < 0.9,
            'created_at': fake.date_time_between(start_date='-2y', end_date='now'),
            'updated_at': fake.date_time_between(start_date='-1y', end_date='now')
        })

    return pd.DataFrame(related_records)

def generate_guarantors(n, loans_df, persons_df):
    """Generate guarantor records for some loans"""
    guarantor_records = []

    # Check if loans_df is empty
    if len(loans_df) == 0:
        print("Warning: No loans data available for guarantors")
        return pd.DataFrame()

    # Check available columns
    # print(f"Available columns in loans_df: {loans_df.columns.tolist()}")

    # Find amount column
    amount_column = None
    for col in ['original_amount', 'loan_amount', 'amount', 'loan_size', 'principal']:
        if col in loans_df.columns:
            amount_column = col
            break

    # Find product type column
    product_column = None
    for col in ['product_type', 'loan_type', 'product', 'type']:
        if col in loans_df.columns:
            product_column = col
            break

    # Select loans that might need guarantors
    if amount_column and product_column:
        # Select loans based on amount and product type
        eligible_loans = loans_df[
            (loans_df[amount_column] > 50000) |
            (loans_df[product_column].isin(['Business Loan', 'Mortgage']))
        ]
    elif amount_column:
        # Select loans based on amount only
        eligible_loans = loans_df[loans_df[amount_column] > 50000]
    else:
        # If no criteria columns found, take a random sample
        sample_size = min(n * 5, len(loans_df))  # Sample more to get enough guarantors
        eligible_loans = loans_df.sample(sample_size)

    # Generate guarantors for eligible loans
    for idx, loan in eligible_loans.iterrows():
        if np.random.random() < 0.2:  # 20% of eligible loans have guarantors
            # Find potential guarantors (different person, good credit)
            if 'personid' in loan:
                borrower_id = loan['personid']
            elif 'person_id' in loan:
                borrower_id = loan['person_id']
            elif 'customer_id' in loan:
                borrower_id = loan['customer_id']
            else:
                borrower_id = np.random.randint(1, len(persons_df))

            # Find suitable guarantors
            potential_guarantors = persons_df[
                (persons_df['personid'] != borrower_id) &
                (persons_df['credit_score'] > 700)
            ]

            # If no suitable guarantors based on credit score, relax criteria
            if len(potential_guarantors) == 0:
                potential_guarantors = persons_df[
                    (persons_df['personid'] != borrower_id) &
                    (persons_df['credit_score'] > 650)
                ]

            # If still no suitable guarantors, take any other person
            if len(potential_guarantors) == 0:
                potential_guarantors = persons_df[persons_df['personid'] != borrower_id]

            if len(potential_guarantors) > 0:
                guarantor = potential_guarantors.sample(1).iloc[0]

                # Get loan amount
                if amount_column and amount_column in loan:
                    guarantee_amount = loan[amount_column]
                else:
                    guarantee_amount = np.random.uniform(10000, 100000)

                # Get loan id
                if 'loanid' in loan:
                    loan_id = loan['loanid']
                elif 'loan_id' in loan:
                    loan_id = loan['loan_id']
                elif 'id' in loan:
                    loan_id = loan['id']
                else:
                    loan_id = idx

                # Get dates
                if 'origination_date' in loan:
                    start_date = loan['origination_date']
                elif 'start_date' in loan:
                    start_date = loan['start_date']
                elif 'funding_date' in loan:
                    start_date = loan['funding_date']
                else:
                    start_date = datetime.now() - timedelta(days=np.random.randint(30, 365))

                if 'maturity_date' in loan:
                    end_date = loan['maturity_date']
                elif 'end_date' in loan:
                    end_date = loan['end_date']
                else:
                    end_date = start_date + timedelta(days=365 * np.random.randint(1, 5))

                # Ensure dates are datetime objects
                if not isinstance(start_date, datetime):
                    try:
                        start_date = pd.to_datetime(start_date)
                    except:
                        start_date = datetime.now() - timedelta(days=np.random.randint(30, 365))

                if not isinstance(end_date, datetime):
                    try:
                        end_date = pd.to_datetime(end_date)
                    except:
                        end_date = start_date + timedelta(days=365 * np.random.randint(1, 5))

                guarantor_records.append({
                    'guarantor_id': len(guarantor_records) + 1,
                    'loanid': loan_id,
                    'guarantor_personid': guarantor['personid'],
                    'borrower_personid': borrower_id,
                    'guarantee_amount': round(guarantee_amount, 2),
                    'guarantee_type': np.random.choice(['Full', 'Partial', 'Limited'],
                                                     p=[0.6, 0.3, 0.1]),
                    'guarantee_start_date': start_date,
                    'guarantee_end_date': end_date,
                    'guarantee_status': np.random.choice(['Active', 'Released', 'Called'],
                                                       p=[0.8, 0.15, 0.05]),
                    'created_at': start_date,
                    'updated_at': datetime.now()
                })

                if len(guarantor_records) >= n:
                    break

    # If we don't have enough guarantors, create some synthetic ones
    while len(guarantor_records) < n:
        # Select random loan and person
        if len(loans_df) > 0:
            loan = loans_df.sample(1).iloc[0]
            loan_id = loan.get('loanid', loan.get('loan_id', loan.get('id', len(guarantor_records))))
        else:
            loan_id = len(guarantor_records) + 1

        # Select two different persons
        if len(persons_df) >= 2:
            two_persons = persons_df.sample(2)
            borrower = two_persons.iloc[0]
            guarantor = two_persons.iloc[1]
        else:
            # Create synthetic IDs if not enough persons
            borrower_id = 1
            guarantor_id = 2

        guarantor_records.append({
            'guarantor_id': len(guarantor_records) + 1,
            'loanid': loan_id,
            'guarantor_personid': guarantor['personid'] if 'guarantor' in locals() else guarantor_id,
            'borrower_personid': borrower['personid'] if 'borrower' in locals() else borrower_id,
            'guarantee_amount': round(np.random.uniform(10000, 100000), 2),
            'guarantee_type': np.random.choice(['Full', 'Partial', 'Limited'],
                                             p=[0.6, 0.3, 0.1]),
            'guarantee_start_date': datetime.now() - timedelta(days=np.random.randint(30, 365)),
            'guarantee_end_date': datetime.now() + timedelta(days=np.random.randint(30, 365*3)),
            'guarantee_status': np.random.choice(['Active', 'Released', 'Called'],
                                               p=[0.8, 0.15, 0.05]),
            'created_at': datetime.now() - timedelta(days=np.random.randint(1, 30)),
            'updated_at': datetime.now()
        })

    return pd.DataFrame(guarantor_records[:n])

def generate_collateral(n, loans_df):
    """Generate collateral records for secured loans"""
    collateral_records = []

    # Check if loans_df is empty
    if len(loans_df) == 0:
        print("Warning: No loans data available for collateral")
        return pd.DataFrame()

    # Check available columns
    # print(f"Available columns in loans_df: {loans_df.columns.tolist()}")

    # Find product type column
    product_column = None
    for col in ['product_type', 'loan_type', 'product', 'type', 'loan_product']:
        if col in loans_df.columns:
            product_column = col
            break

    # Find amount column - DEFINE HERE for the whole function
    amount_column = None
    for col in ['original_amount', 'loan_amount', 'amount', 'loan_size', 'principal']:
        if col in loans_df.columns:
            amount_column = col
            break

    # Select loans that should have collateral
    if product_column:
        secured_loans = loans_df[loans_df[product_column].isin(['Mortgage', 'Auto Loan', 'Business Loan'])]
    else:
        # If no product column found, take a sample based on loan amount
        if amount_column:
            # Assume larger loans are more likely to be secured
            secured_loans = loans_df[loans_df[amount_column] > 50000]
        else:
            # Take a random sample if no criteria columns found
            sample_size = min(n * 2, len(loans_df))  # Sample more to get enough collateral
            secured_loans = loans_df.sample(sample_size)

    # Generate collateral records
    for idx, loan in secured_loans.iterrows():
        if len(collateral_records) >= n:
            break

        # Determine collateral type based on product type or loan amount
        if product_column and product_column in loan:
            product_type = loan[product_column]
            if product_type == 'Mortgage':
                collateral_type = 'Real Estate'
                description = f"{np.random.choice(['Single Family', 'Condo', 'Townhouse', 'Multi-family'])} Property"
                value_multiplier = np.random.uniform(1.1, 1.5)
            elif product_type == 'Auto Loan':
                collateral_type = 'Vehicle'
                description = f"{np.random.choice(['New', 'Used'])} Vehicle"
                value_multiplier = np.random.uniform(1.05, 1.3)
            elif product_type == 'Business Loan':
                collateral_type = np.random.choice(['Equipment', 'Inventory', 'Accounts Receivable', 'Real Estate'])
                description = f"Business {collateral_type}"
                value_multiplier = np.random.uniform(1.2, 2.0)
            else:
                collateral_type = np.random.choice(['Real Estate', 'Vehicle', 'Equipment', 'Other'])
                description = f"{collateral_type} Asset"
                value_multiplier = np.random.uniform(1.1, 1.5)
        else:
            # Determine collateral type based on loan amount
            if amount_column and loan[amount_column] > 100000:
                collateral_type = 'Real Estate'
                description = "Property"
                value_multiplier = np.random.uniform(1.2, 1.8)
            elif amount_column and loan[amount_column] > 20000:
                collateral_type = 'Vehicle'
                description = "Vehicle"
                value_multiplier = np.random.uniform(1.1, 1.4)
            else:
                collateral_type = np.random.choice(['Equipment', 'Inventory', 'Other'])
                description = f"{collateral_type} Asset"
                value_multiplier = np.random.uniform(1.1, 1.5)

        # Get loan amount
        loan_amount = 0
        if amount_column and amount_column in loan:
            loan_amount = loan[amount_column]
        else:
            loan_amount = np.random.uniform(10000, 500000)

        collateral_value = loan_amount * value_multiplier

        # Get loan id
        if 'loanid' in loan:
            loan_id = loan['loanid']
        elif 'loan_id' in loan:
            loan_id = loan['loan_id']
        elif 'id' in loan:
            loan_id = loan['id']
        else:
            loan_id = idx

        # Get dates
        if 'origination_date' in loan:
            valuation_date = loan['origination_date'] - timedelta(days=np.random.randint(1, 30))
        elif 'start_date' in loan:
            valuation_date = loan['start_date'] - timedelta(days=np.random.randint(1, 30))
        elif 'funding_date' in loan:
            valuation_date = loan['funding_date'] - timedelta(days=np.random.randint(1, 30))
        else:
            valuation_date = datetime.now() - timedelta(days=np.random.randint(60, 365))

        # Ensure valuation_date is datetime
        if not isinstance(valuation_date, datetime):
            try:
                valuation_date = pd.to_datetime(valuation_date)
            except:
                valuation_date = datetime.now() - timedelta(days=np.random.randint(60, 365))

        collateral_records.append({
            'collateral_id': len(collateral_records) + 1,
            'loanid': loan_id,
            'collateral_type': collateral_type,
            'description': description,
            'original_value': round(collateral_value, 2),
            'current_value': round(collateral_value * np.random.uniform(0.9, 1.1), 2),
            'valuation_date': valuation_date.date() if hasattr(valuation_date, 'date') else valuation_date,
            'valuation_method': np.random.choice(['Appraisal', 'Market Value', 'Book Value', 'Third Party']),
            'ltv_ratio': round(loan_amount / collateral_value, 3),
            'insurance_required': True,
            'insurance_provider': np.random.choice(['Provider A', 'Provider B', 'Provider C', 'Other']),
            'insurance_policy_number': fake.bothify(text='POL-########'),
            'lien_position': np.random.choice(['First', 'Second'], p=[0.9, 0.1]),
            'status': np.random.choice(['Active', 'Released'], p=[0.9, 0.1]),
            'created_at': valuation_date,
            'updated_at': datetime.now()
        })

    # If we don't have enough collateral records, create synthetic ones
    while len(collateral_records) < n:
        loan_id = len(collateral_records) + 1
        loan_amount = np.random.uniform(10000, 500000)
        collateral_value = loan_amount * np.random.uniform(1.1, 2.0)
        valuation_date = datetime.now() - timedelta(days=np.random.randint(30, 365))

        collateral_records.append({
            'collateral_id': len(collateral_records) + 1,
            'loanid': loan_id,
            'collateral_type': np.random.choice(['Real Estate', 'Vehicle', 'Equipment', 'Inventory', 'Other']),
            'description': f"Asset #{len(collateral_records) + 1}",
            'original_value': round(collateral_value, 2),
            'current_value': round(collateral_value * np.random.uniform(0.9, 1.1), 2),
            'valuation_date': valuation_date.date(),
            'valuation_method': np.random.choice(['Appraisal', 'Market Value', 'Book Value', 'Third Party']),
            'ltv_ratio': round(loan_amount / collateral_value, 3),
            'insurance_required': True,
            'insurance_provider': np.random.choice(['Provider A', 'Provider B', 'Provider C', 'Other']),
            'insurance_policy_number': fake.bothify(text='POL-########'),
            'lien_position': np.random.choice(['First', 'Second'], p=[0.9, 0.1]),
            'status': np.random.choice(['Active', 'Released'], p=[0.9, 0.1]),
            'created_at': valuation_date,
            'updated_at': datetime.now()
        })

    return pd.DataFrame(collateral_records[:n])

def generate_synthetic_database():
    """Main function to generate all synthetic data"""
    print("Starting synthetic data generation...")
    print(f"Target database size: {TARGET_FINAL_SIZE_GB}GB")
    print(f"Scaling factor: {scaling_factor:.2f}")

    # Create output directory
    os.makedirs("./synthetic_data", exist_ok=True)

    synthetic_data = {}

    def timed_step(step_name, func, *args):
        start = time.time()
        result = func(*args)
        duration = time.time() - start
        print(f"{step_name} completed in {duration:.2f} seconds.")
        return result

    # Generate data in order of dependencies
    synthetic_data['persons'] = timed_step("1. Generating persons", generate_persons, TARGET_ROW_COUNTS['persons'])
    synthetic_data['person_employment'] = timed_step("2. Generating person employment", generate_person_employment, TARGET_ROW_COUNTS['person_employment'], synthetic_data['persons'])
    synthetic_data['applications'] = timed_step("3. Generating applications", generate_applications, TARGET_ROW_COUNTS['applications'], synthetic_data['persons'], synthetic_data['person_employment'])
    synthetic_data['application_financial'] = timed_step("4. Generating application financial data", generate_application_financial, TARGET_ROW_COUNTS['application_financial'], synthetic_data['applications'], synthetic_data['persons'], synthetic_data['person_employment'])
    synthetic_data['credit_scoring'] = timed_step("5. Generating credit scoring", generate_credit_scoring, TARGET_ROW_COUNTS['credit_scoring'], synthetic_data['applications'], synthetic_data['persons'], synthetic_data['application_financial'])
    synthetic_data['loans'] = timed_step("6. Generating loans", generate_loans, TARGET_ROW_COUNTS['loans'], synthetic_data['applications'])
    synthetic_data['loan_payments'] = timed_step("7. Generating loan payments", generate_loan_payments, TARGET_ROW_COUNTS['loan_payments'], synthetic_data['loans'])
    synthetic_data['transactions'] = timed_step("8. Generating transactions", generate_transactions, TARGET_ROW_COUNTS['transactions'], synthetic_data['persons'])
    synthetic_data['transactions_p2p'] = timed_step("9. Generating P2P transactions", generate_transactions_p2p, TARGET_ROW_COUNTS['transactions_p2p'], synthetic_data['persons'])
    synthetic_data['vwlob'] = timed_step("10. Generating loan observations (vwlob)", generate_vwlob, synthetic_data['loans']["origination_date"].max(), synthetic_data['loans'])
    synthetic_data['wof'] = timed_step("11. Generating write-offs", generate_wof, TARGET_ROW_COUNTS['wof'], synthetic_data['loans'])
    synthetic_data['related_persons'] = timed_step("12. Generating related persons", generate_related_persons, TARGET_ROW_COUNTS['related_persons'], synthetic_data['persons'])
    synthetic_data['guarantors'] = timed_step("13. Generating guarantors", generate_guarantors, TARGET_ROW_COUNTS['guarantors'], synthetic_data['loans'], synthetic_data['persons'])
    synthetic_data['collateral'] = timed_step("14. Generating collateral", generate_collateral, TARGET_ROW_COUNTS['collateral'], synthetic_data['loans'])

    # Calculate actual size
    print("\n=== Table Statistics ===")
    total_size_bytes = 0
    for name, df in synthetic_data.items():
        size_bytes = df.memory_usage(deep=True).sum()
        size_mb = size_bytes / (1024 ** 2)
        total_size_bytes += size_bytes
        print(f"{name:<25}: {len(df):>10,} rows, {size_mb:>8.1f} MB")

    total_size_gb = total_size_bytes / (1024 ** 3)
    print(f"\n{'TOTAL':<25}: {total_size_gb:>8.2f} GB")

    # Save to pickle
    # output_file = './synthetic_data/synthetic_database.pkl'
    # print(f"\nSaving to {output_file}...")
    # with open(output_file, 'wb') as f:
    #     pickle.dump(synthetic_data, f)

    print("\nData generation complete!")
    return synthetic_data



In [4]:
synthetic_data = generate_synthetic_database()

Starting synthetic data generation...
Target database size: 0.1GB
Scaling factor: 0.01
1. Generating persons completed in 0.39 seconds.
2. Generating person employment completed in 0.56 seconds.
3. Generating applications completed in 0.14 seconds.
4. Generating application financial data completed in 0.30 seconds.
5. Generating credit scoring completed in 0.02 seconds.
6. Generating loans completed in 0.01 seconds.
7. Generating loan payments completed in 0.92 seconds.
8. Generating transactions completed in 5.70 seconds.
9. Generating P2P transactions completed in 0.36 seconds.
10. Generating loan observations (vwlob) completed in 23.55 seconds.
11. Generating write-offs completed in 0.00 seconds.
12. Generating related persons completed in 0.15 seconds.
13. Generating guarantors completed in 0.08 seconds.
14. Generating collateral completed in 0.03 seconds.

=== Table Statistics ===
persons                  :      1,212 rows,      1.3 MB
person_employment        :      1,769 rows,  

In [14]:
synthetic_data["vwlob"]

,vwlob_id,loanid,appid,personid,observation_date,days_on_book,months_on_book,current_balance,overdue_days,payment_status,ever_delinquent,max_delinquency,loan_status,created_at,updated_at
0,1,1277,2324,893,2023-05-07,0,0,26395.68,0,Current,False,0,Current,2023-05-07 05:37:12,2023-05-07 05:37:12
1,2,1277,2324,893,2023-05-08,1,0,26377.48,0,Current,False,0,Current,2023-05-08 05:37:12,2023-05-08 05:37:12
2,3,1277,2324,893,2023-05-09,2,0,26359.27,0,Current,False,0,Current,2023-05-09 05:37:12,2023-05-09 05:37:12
3,4,1277,2324,893,2023-05-10,3,0,26341.07,0,Current,False,0,Current,2023-05-10 05:37:12,2023-05-10 05:37:12
4,5,1277,2324,893,2023-05-11,4,0,26322.86,0,Current,False,0,Current,2023-05-11 05:37:12,2023-05-11 05:37:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480296,480297,1666,3017,1206,2025-05-23,527,17,19477.40,0,Current,False,0,Current,2025-05-23 05:37:12,2025-05-23 05:37:12
480297,480298,1667,3018,595,2025-05-23,392,13,31716.93,0,Current,False,0,Current,2025-05-23 05:37:12,2025-05-23 05:37:12
480298,480299,1670,3022,530,2025-05-23,279,9,17835.63,0,Current,False,0,Current,2025-05-23 05:37:12,2025-05-23 05:37:12
480299,480300,1674,3028,295,2025-05-23,15,0,33900.16,0,Current,False,0,Pending,2025-05-23 05:37:12,2025-05-23 05:37:12
